In [1]:
import boto3
import json

### Message and Prompt Formatter

In [2]:
def format_llama_messages(messages):
    """Format messages for Llama-2 chat models.

    The model only supports 'system', 'user', and 'assistant' roles, starting with 'system', then 'user', and
    alternating (u/a/u/a/u...). The last message must be from 'user'.
    """
    prompt = []

    if messages[0]["role"] == "system":
        content = "".join(["<<SYS>>\n", messages[0]["content"], "\n<</SYS>>\n\n", messages[1]["content"]])
        messages = [{"role": messages[1]["role"], "content": content}] + messages[2:]

    for user, answer in zip(messages[::2], messages[1::2]):
        prompt.extend(["<s>", "[INST] ", (user["content"]).strip(), " [/INST] ", (answer["content"]).strip(), "</s>"])

    prompt.extend(["<s>", "[INST] ", (messages[-1]["content"]).strip(), " [/INST] "])

    return "".join(prompt)

### User and System Prompts

In [3]:
user_prompt = "Write a poem on machine learning."
system_prompt = "You a Shakespearian poet"

### Formatting the Prompt

In [4]:
all_messages = [{"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}]
formatted_prompt = format_llama_messages(all_messages)


### Formatting the Payload

In [5]:
max_tokens_to_generate = 256
payload = {"prompt": formatted_prompt, 
           "max_gen_len": max_tokens_to_generate, 
           "temperature": 0.1, 
           "top_p": 0.9
           }

In [6]:
bedrock = boto3.client(service_name="bedrock-runtime")

### Formatting and Invocating

In [7]:
modelId = 'meta.llama2-70b-chat-v1'
accept = 'application/json'
contentType = 'application/json'

response = bedrock.invoke_model(body=json.dumps(payload),
                                modelId=modelId,
                                accept=accept,
                                contentType=contentType
                                )

### Response and Output

In [8]:
response_body=json.loads(response.get("body").read())
response_text=response_body['generation']
print(response_text)

 In the digital realm, a new magic doth abide,
A marvel of man's mind, a wonder to abide,
'Tis machine learning, a science most divine,
That doth bring forth knowledge, with algorithms so fine.

With data as its fuel, it doth learn and grow,
A neural network of wisdom, for all to know,
It doth predict and classify, with such grace,
A true marvel of science, in this digital place.

The languages it speaks, are many and vast,
Python, R, and Java, to name a few at last,
It doth consume data, like a starving beast,
And from it, knowledge doth spring, like a bless'd feast.

The wonders it doth work, are many to behold,
Image recognition, speech, and natural language to unfold,
It doth predict the future, with such precision,
A true marvel of man's mind, this machine obsession.

But beware, dear friends, for with great power comes great responsibility,
The line between good and evil, can quickly become hazy,

